In [10]:
# Вариант 9
import numpy as np
import matplotlib.pyplot as plt

# Параметры задачи
q_coeff = 1.0
p_coeff = 3.0

a_coeff = 1.0
b_coeff = 1.0
c_coeff = -1.0

f = lambda x, t: np.sin(x) * np.exp(-t)

# Граничные условия: alpha * u_x + beta * u = gamma
alpha0 = 0
beta0 = 1
gamma0 = lambda t: np.exp(-t)

alphal = 0
betal = 1
gammal = lambda t: -np.exp(-t)

# Начальное условие
psi1 = lambda x: np.cos(x)
psi2 = lambda x: -np.cos(x)

dpsi1 = lambda x: -np.sin(x)
ddpsi1 = lambda x: -np.cos(x)


L = np.pi
T_max = 10.0

# Аналитическое решение для проверки
analytical_solution = lambda x, t: np.exp(-t) * np.cos(x)


# Сеточные параметры
L_count = 100
X = np.linspace(0, L, L_count + 1)
h = L / L_count

T_count = 10000
T = np.linspace(0, T_max, T_count + 1)
tau = T_max / T_count


In [11]:
# Решение неявной схемой
def solve_hyperbolic_implicit(boundary_approx_type="two_point_second_order"):
    u_num = np.zeros((len(T), len(X)))
    u_num[0, :] = psi1(X)
    u_num[1, :] = psi1(X) + psi2(X) * tau + \
         (a_coeff * ddpsi1(X) + b_coeff * dpsi1(X) + c_coeff * psi1(X) + f(X, T[0]) - p_coeff * psi2(X)) * tau**2 / 2
    
    N = len(X) - 1
    
    for n in range(1, len(T) - 1):
        u_prev = u_num[n, :]
        u_prev_prev = u_num[n - 1, :]
        u_next = np.zeros_like(u_prev)

        A = np.zeros(N + 1)
        B = np.zeros(N + 1)
        C = np.zeros(N + 1)
        D = np.zeros(N + 1)

        for i in range(1, N):
            A[i] = -a_coeff / h ** 2 + b_coeff / (2 * h)
            B[i] = q_coeff / tau ** 2 + p_coeff / (2 * tau) + 2 * a_coeff / h ** 2 - c_coeff
            C[i] = -a_coeff / h ** 2 - b_coeff / (2 * h)
            D[i] = f(X[i], T[n + 1]) - u_prev_prev[i] * (q_coeff / tau ** 2 - p_coeff / (2 * tau)) - u_prev[i] * (-2 * q_coeff / tau ** 2)

        t_next = T[n + 1]
        
        if boundary_approx_type == "two_point_first_order":
            A[0] = 0
            B[0] = beta0 - alpha0 / h
            C[0] = alpha0 / h
            D[0] = gamma0(t_next)
            
            A[N] = -alphal / h
            B[N] = alphal / h + betal
            C[N] = 0
            D[N] = gammal(t_next)
 
        elif boundary_approx_type == "three_point_second_order":
            coeff0 = (-alpha0 / 2 / h) / C[1]
            A[0] = 0
            B[0] = (alpha0 * (-3/(2*h)) + beta0) - A[1] * coeff0
            C[0] = (alpha0 * (4/(2*h))) - B[1] * coeff0
            D[0] = gamma0(t_next) - D[1] * coeff0
            
            coeffl = (alphal / 2 / h) / A[N - 1]
            A[N] = -alphal * (4/(2*h)) - B[N - 1] * coeffl
            B[N] = alphal * (3/(2*h)) + betal - C[N - 1] * coeffl
            C[N] = 0
            D[N] = gammal(t_next) - D[N - 1] * coeffl
            
        elif boundary_approx_type == "two_point_second_order":
            A[0] = 0
            B[0] = alpha0 * ((c_coeff * h ** 2 / 2 / a_coeff - 1 - q_coeff * h ** 2 / 2 / a_coeff / tau ** 2 - p_coeff * h ** 2 / 4 / a_coeff / tau) / (h - b_coeff * h ** 2 / 2 / a_coeff)) + beta0
            C[0] = alpha0 * (1 / (h - b_coeff * h ** 2 / 2 / a_coeff))
            D[0] = gamma0(t_next) - alpha0 * (u_prev[0] * (q_coeff * h ** 2 / a_coeff / tau ** 2) + u_prev_prev[0] * (-q_coeff * h ** 2 / 2 / a_coeff / tau ** 2 + p_coeff * h ** 2 / 4 / a_coeff / tau) + f(X[0], t_next) * (h ** 2 / 2 / a_coeff)) / (h - b_coeff * h ** 2 / 2 / a_coeff)
            
            A[N] = alphal * (-1 / (h + b_coeff * h ** 2 / 2 / a_coeff))
            B[N] = alphal * ((1 - c_coeff * h ** 2 / 2 / a_coeff + q_coeff * h ** 2 / 2 / a_coeff / tau ** 2 + p_coeff * h ** 2 / 4 / a_coeff / tau) / (h + b_coeff * h ** 2 / 2 / a_coeff)) + betal
            C[N] = 0
            D[N] = gammal(t_next) - alphal * (u_prev[N] * (-q_coeff * h ** 2 / a_coeff / tau ** 2) + u_prev_prev[N] * (q_coeff * h ** 2 / 2 / a_coeff / tau ** 2 - p_coeff * h ** 2 / 4 / a_coeff / tau) + f(X[N], t_next) * (-1 *h ** 2 / 2 / a_coeff)) / (h + b_coeff * h ** 2 / 2 / a_coeff)
        
        # Метод прогонки
        alpha_prop = np.zeros(N + 1)
        beta_prop = np.zeros(N + 1)

        if abs(B[0]) < 1e-12:
            raise ValueError("Деление на ноль в методе прогонки")
        
        alpha_prop[0] = -C[0] / B[0]
        beta_prop[0] = D[0] / B[0]

        for i in range(1, N + 1):
            denominator = B[i] + A[i] * alpha_prop[i - 1]
            if abs(denominator) < 1e-12:
                raise ValueError("Деление на ноль в методе прогонки")
            if i < N:
                alpha_prop[i] = -C[i] / denominator
            beta_prop[i] = (D[i] - A[i] * beta_prop[i - 1]) / denominator

        u_next[N] = beta_prop[N]

        for i in range(N - 1, -1, -1):
            u_next[i] = alpha_prop[i] * u_next[i + 1] + beta_prop[i]

        u_num[n + 1, :] = u_next

    return u_num


def solve_hyperbolic_explicit(boundary_approx_type="two_point_second_order"):
    sigma = a_coeff * tau ** 2 / h ** 2
    if sigma >= 1.0:
        print(f"ВНИМАНИЕ: Число σ = {sigma:.3f} >= 1.0, схема может быть неустойчива!")

    u_num = np.zeros((len(T), len(X)))
    u_num[0, :] = psi1(X)
    u_num[1, :] = psi1(X) + psi2(X) * tau + \
         (a_coeff * ddpsi1(X) + b_coeff * dpsi1(X) + c_coeff * psi1(X) + f(X, T[0]) - p_coeff * psi2(X)) * tau**2 / 2
    
    N = len(X) - 1
    
    for n in range(1, len(T) - 1):
        u_prev = u_num[n, :]
        u_prev_prev = u_num[n - 1, :]
        u_next = np.zeros_like(u_prev)
        
        for i in range(1, N):
            u_next[i] = (u_prev_prev[i] * (p_coeff / 2 / tau - q_coeff / tau ** 2) + \
                         u_prev[i - 1] * (a_coeff / h ** 2 - b_coeff / 2 / h) + \
                         u_prev[i] * (2 * q_coeff / tau ** 2 - 2 * a_coeff / h ** 2 + c_coeff) + \
                         u_prev[i + 1] * (a_coeff / h ** 2 + b_coeff / 2 / h) + \
                         f(X[i], T[n])) / (q_coeff / tau ** 2 + p_coeff / 2 / tau)
        
        t_next = T[n + 1]
        
        if approx_type == "two_point_first_order":
            u_next[0] = (gamma0(t_next) - (alpha0 / h) * u_next[1]) / (beta0 - (alpha0 / h))
            u_next[N] = (gammal(t_next) + (alphal / h) * u_next[N - 1]) / (betal + (alphal / h))
            
        elif approx_type == "three_point_second_order":
            u_next[0] = (gamma0(t_next) - u_next[1] * (4 * alpha0 / 2 / h) + u_next[2] * (alpha0 / 2 / h )) / \
                        ((-3 * alpha0 / 2 / h) + beta0)
            u_next[N] = (gammal(t_next) - u_next[N - 2] * (alphal / 2 / h) - u_next[N - 1] * (-4 * alphal / 2 / h)) / \
                        ((3 * alphal / 2 / h) + betal)

        elif approx_type == "two_point_second_order":
            b0 = alpha0 * ((c_coeff * h ** 2 / 2 / a_coeff - 1 - q_coeff * h ** 2 / 2 / a_coeff / tau ** 2 - p_coeff * h ** 2 / 4 / a_coeff / tau) / (h - b_coeff * h ** 2 / 2 / a_coeff)) + beta0
            c0 = alpha0 * (1 / (h - b_coeff * h ** 2 / 2 / a_coeff))
            d0 = gamma0(t_next) - alpha0 * (u_prev[0] * (q_coeff * h ** 2 / a_coeff / tau ** 2) + u_prev_prev[0] * (-q_coeff * h ** 2 / 2 / a_coeff / tau ** 2 + p_coeff * h ** 2 / 4 / a_coeff / tau) + f(X[0], t_next) * (h ** 2 / 2 / a_coeff)) / (h - b_coeff * h ** 2 / 2 / a_coeff)
            
            al = alphal * (-1 / (h + b_coeff * h ** 2 / 2 / a_coeff))
            bl = alphal * ((1 - c_coeff * h ** 2 / 2 / a_coeff + q_coeff * h ** 2 / 2 / a_coeff / tau ** 2 + p_coeff * h ** 2 / 4 / a_coeff / tau) / (h + b_coeff * h ** 2 / 2 / a_coeff)) + betal
            dl = gammal(t_next) - alphal * (u_prev[N] * (-q_coeff * h ** 2 / a_coeff / tau ** 2) + u_prev_prev[N] * (q_coeff * h ** 2 / 2 / a_coeff / tau ** 2 - p_coeff * h ** 2 / 4 / a_coeff / tau) + f(X[N], t_next) * (-1 *h ** 2 / 2 / a_coeff)) / (h + b_coeff * h ** 2 / 2 / a_coeff)

            u_next[0] = (d0 - c0 * u_next[1]) / b0
            u_next[N] = (dl - al * u_next[N - 1]) / bl 

        u_num[n + 1, :] = u_next

    return u_num




schemes = [
    'Явная схема',
    'Неявная схема',
]

boundary_approximations = [
    'two_point_first_order',
    'three_point_second_order',
    'two_point_second_order', 
]

results = {}

for scheme_name in schemes:
    for approx_type in boundary_approximations:
        key = f"{scheme_name}_{approx_type}"
        print(f"Вычисление: {key}")
        try:
            if scheme_name == 'Неявная схема':
                u_num = solve_hyperbolic_implicit(approx_type)
            elif scheme_name == 'Явная схема':
                u_num = solve_hyperbolic_explicit(approx_type)
            else:
                raise ValueError(f"Неизвестная схема: {scheme_name}")
            errors = {}
            time_indices = [len(T) // 4, len(T) // 2, 3 * len(T) // 4, len(T) - 1]
            for idx in time_indices:
                if idx < len(u_num):
                    u_analytical = analytical_solution(X, T[idx])
                    error = np.abs(u_num[idx, :] - u_analytical)
                    errors[T[idx]] = np.max(error)
            results[key] = {
                'solution': u_num,
                'errors': errors
            }
        except Exception as e:
            print(f"Ошибка при вычислении {key}: {e}")


Вычисление: Явная схема_two_point_first_order
Вычисление: Явная схема_three_point_second_order
Вычисление: Явная схема_two_point_second_order
Вычисление: Неявная схема_two_point_first_order
Вычисление: Неявная схема_three_point_second_order
Вычисление: Неявная схема_two_point_second_order


In [12]:
def plot_results_interactive(results, X, T):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import numpy as np

    # Моменты времени
    time_indices = [
        0,
        len(T) // 3,
        2 * len(T) // 3,
        len(T) - 1
    ]
    time_labels = [f"t = {T[i]:.2f}" for i in time_indices]

    # Цвета для уникальности
    base_colors = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
        '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
        '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5'
    ]
    color_map = {key: base_colors[i % len(base_colors)] for i, key in enumerate(results.keys())}

    # ---------- 1. Графики решений ----------
    fig_sol = make_subplots(
        rows=2, cols=2,
        subplot_titles=time_labels,
        horizontal_spacing=0.08,
        vertical_spacing=0.12
    )

    for plot_idx, t_idx in enumerate(time_indices):
        row = (plot_idx // 2) + 1
        col = (plot_idx % 2) + 1
        t_val = T[t_idx]

        # Аналитическое решение — показываем в легенде только в первом графике
        u_anal = analytical_solution(X, t_val)
        show_legend_analytical = (plot_idx == 0)  # Только один раз
        fig_sol.add_trace(
            go.Scatter(
                x=X, y=u_anal, mode='lines',
                name='Аналитическое',
                line=dict(color='black', dash='dash', width=3),
                showlegend=show_legend_analytical,
                legendgroup='analytical'  # ← группа
            ),
            row=row, col=col
        )

        # Численные решения
        for key, data in results.items():
            u_num = data['solution'][t_idx, :]
            # Показываем в легенде только в первом графике (plot_idx == 0)
            show_legend = (plot_idx == 0)
            fig_sol.add_trace(
                go.Scatter(
                    x=X, y=u_num, mode='lines',
                    name=key.replace('_', ' ')
                               .replace('two point', '2-point')
                               .replace('three point', '3-point'),
                    line=dict(width=2, color=color_map[key]),
                    legendgroup=key,  # ← группа
                    showlegend=show_legend,  # ← ключевое: только один раз!
                    visible=True
                ),
                row=row, col=col
            )

    fig_sol.update_layout(
        title="Сравнение решений (все схемы и аппроксимации)",
        title_x=0.5,
        height=800,
        width=1400,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.02,
            font=dict(size=10),
            itemclick="toggle",        # клик переключает ТОЛЬКО эту линию
            itemdoubleclick="toggle",
            bgcolor="rgba(255,255,255,0.85)"
        ),
        hovermode='x unified',
        margin=dict(r=200)
    )

    fig_sol.update_xaxes(title_text="x", showgrid=True, gridcolor='LightGray')
    fig_sol.update_yaxes(title_text="u(x,t)", showgrid=True, gridcolor='LightGray')
    fig_sol.show()

    # ---------- 2. График ошибок ----------
    fig_err = go.Figure()

    for key, data in results.items():
        if 'errors' in data and data['errors']:
            times = list(data['errors'].keys())
            max_errors = list(data['errors'].values())
            fig_err.add_trace(go.Scatter(
                x=times, y=max_errors,
                mode='lines+markers',
                name=key.replace('_', ' ')
                           .replace('two point', '2-point')
                           .replace('three point', '3-point'),
                line=dict(width=2, color=color_map[key]),
                marker=dict(size=4)
            ))

    fig_err.update_layout(
        title="Максимальная ошибка во времени",
        xaxis_title="Время t",
        yaxis_title="Максимальная ошибка (L∞)",
        yaxis_type="log",
        height=600,
        width=1200,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.02,
            font=dict(size=10),
            itemclick="toggle",
            itemdoubleclick="toggle",
            bgcolor="rgba(255,255,255,0.85)"
        ),
        hovermode='x unified',
        margin=dict(r=200),
        xaxis=dict(showgrid=True, gridcolor='LightGray'),
        yaxis=dict(showgrid=True, gridcolor='LightGray')
    )

    fig_err.show()
    
if __name__ == "__main__":
    plot_results_interactive(results, X, T)

    print("\n" + "=" * 60)
    print("ТАБЛИЦА ПОГРЕШНОСТЕЙ")
    print("=" * 60)

    for scheme_name in results.keys():
        print(f"\n{scheme_name}:")
        print("Время\tПогрешность")
        for time, error in results[scheme_name]['errors'].items():
            print(f"{time:.3f}\t{error:.2e}")


ТАБЛИЦА ПОГРЕШНОСТЕЙ

Явная схема_two_point_first_order:
Время	Погрешность
2.500	2.05e-05
5.000	3.30e-06
7.500	3.36e-07
10.000	2.98e-08

Явная схема_three_point_second_order:
Время	Погрешность
2.500	2.05e-05
5.000	3.30e-06
7.500	3.36e-07
10.000	2.98e-08

Явная схема_two_point_second_order:
Время	Погрешность
2.500	2.05e-05
5.000	3.30e-06
7.500	3.36e-07
10.000	2.98e-08

Неявная схема_two_point_first_order:
Время	Погрешность
2.500	6.33e-05
5.000	7.37e-06
7.500	7.09e-07
10.000	6.29e-08

Неявная схема_three_point_second_order:
Время	Погрешность
2.500	6.33e-05
5.000	7.37e-06
7.500	7.09e-07
10.000	6.29e-08

Неявная схема_two_point_second_order:
Время	Погрешность
2.500	6.33e-05
5.000	7.37e-06
7.500	7.09e-07
10.000	6.29e-08
